In [260]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
from catboost import CatBoostClassifier, Pool

In [261]:
root_dir = os.path.dirname(os.getcwd())
display(root_dir)
data_dir = os.path.join(root_dir, 'data')
display(data_dir)
train_path = os.path.join(data_dir, 'train.xlsx')
display(train_path)

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack'

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack\\data'

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack\\data\\train.xlsx'

In [262]:
df = pd.read_excel(train_path)

In [263]:
display(df.head())

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Статус брони,Категория номера,Гостей,Гостиница
0,0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Отмена,Номер «Стандарт»,2,1
1,1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,1
2,2,20221204-16563-171020423,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Активный,Номер «Студия»,2,4
3,3,20230918-7491-223512699,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Активный,Номер «Стандарт»,1,3
4,4,20230529-6634-200121971,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Активный,Номер «Люкс»,4,1


In [264]:
target = df['Дата отмены'].notna().astype(int)
display(target)

0        1
1        0
2        0
3        0
4        0
        ..
26169    0
26170    0
26171    0
26172    1
26173    0
Name: Дата отмены, Length: 26174, dtype: int32

In [265]:
df = df.drop(["Unnamed: 0","№ брони", "Дата отмены","Статус брони"], axis=1)
display(df)

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Номер «Стандарт»,2,1
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Номер «Студия»,2,4
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Номер «Стандарт»,1,3
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Номер «Люкс»,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2023-01-07 17:45:18,2023-03-10 15:00:00,2,2023-03-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,2
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),2023-06-20 17:54:17,2023-06-25 15:00:00,3,2023-06-28 12:00:00,Официальный сайт,Номер «Студия»,3,4
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),2022-05-08 19:24:05,2022-06-24 15:00:00,4,2022-06-28 12:00:00,Официальный сайт,Номер «Стандарт»,2,2
26172,1,6300.0,0,Гарантия банковской картой,2022-02-19 09:55:50,2022-04-27 15:00:00,1,2022-04-28 12:00:00,booking.com,Номер «Стандарт»,2,3


### Train-test split learning

In [266]:
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)
display(train_df.shape, test_df.shape)

(20939, 12)

(5235, 12)

In [267]:
categorical_features = ["Способ оплаты", "Источник","Категория номера"]

In [268]:
train_target = target[train_df.index]
test_target = target.drop(train_df.index)
display(train_target,train_target.shape)
display(test_target,test_target.shape)


5405     0
20995    0
15439    0
19500    0
17112    0
        ..
17888    1
1806     0
3871     0
3791     1
24555    0
Name: Дата отмены, Length: 20939, dtype: int32

(20939,)

9        0
11       0
13       0
16       1
24       1
        ..
26155    0
26156    0
26163    0
26170    0
26172    1
Name: Дата отмены, Length: 5235, dtype: int32

(5235,)

In [269]:
train_pool = Pool(train_df, train_target, cat_features=categorical_features)
test_pool = Pool(test_df, test_target, cat_features=categorical_features)

### Model init with params {epochs = 100 , depth = 6} - will test

In [270]:
model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=True)

In [271]:
model.fit(train_pool)

0:	learn: 0.6413841	total: 20.6ms	remaining: 2.04s
1:	learn: 0.5992745	total: 42ms	remaining: 2.06s
2:	learn: 0.5642641	total: 62.6ms	remaining: 2.02s
3:	learn: 0.5349808	total: 85.6ms	remaining: 2.05s
4:	learn: 0.5104161	total: 107ms	remaining: 2.04s
5:	learn: 0.4892946	total: 131ms	remaining: 2.05s
6:	learn: 0.4634180	total: 154ms	remaining: 2.05s
7:	learn: 0.4419321	total: 168ms	remaining: 1.93s
8:	learn: 0.4236320	total: 185ms	remaining: 1.87s
9:	learn: 0.4073668	total: 208ms	remaining: 1.87s
10:	learn: 0.3944489	total: 229ms	remaining: 1.86s
11:	learn: 0.3829794	total: 254ms	remaining: 1.86s
12:	learn: 0.3728106	total: 277ms	remaining: 1.85s
13:	learn: 0.3643260	total: 299ms	remaining: 1.84s
14:	learn: 0.3570506	total: 329ms	remaining: 1.86s
15:	learn: 0.3508433	total: 354ms	remaining: 1.86s
16:	learn: 0.3448733	total: 380ms	remaining: 1.85s
17:	learn: 0.3400303	total: 400ms	remaining: 1.82s
18:	learn: 0.3362177	total: 419ms	remaining: 1.79s
19:	learn: 0.3328528	total: 436ms	remai

In [272]:
predictions = model.predict(test_pool)
display(predictions,predictions.shape)
display(test_target)
accuracy = (predictions == test_target).mean()
print(f'Accuracy: {accuracy}')
from sklearn.metrics import roc_auc_score
score = roc_auc_score(predictions,test_target)
display(score)

array([0, 0, 0, ..., 0, 0, 1], dtype=int64)

(5235,)

9        0
11       0
13       0
16       1
24       1
        ..
26155    0
26156    0
26163    0
26170    0
26172    1
Name: Дата отмены, Length: 5235, dtype: int32

Accuracy: 0.889207258834766


0.9149239067230767

In [273]:
tdf_path = os.path.join(data_dir, 'test.xlsx')
display(tdf_path)
tdf = pd.read_excel(tdf_path)
display(tdf.head())

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack\\data\\test.xlsx'

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,0,20231129-16563-238946689,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,20221219-7491-174959103,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,2,20221211-6634-172724329,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,3,20230821-6634-212247350,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,4,20230326-6634-189784563,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1


In [274]:
tdf = tdf.drop(["Unnamed: 0","№ брони"], axis=1)
display(tdf)

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11213,1,50200.0,50200,Банк. карта [Кешбэк. МИР]: Эквайринг TravelLin...,2022-03-27 21:30:38,2022-05-07 15:00:00,2,2022-05-09 12:00:00,Официальный сайт,Номер «Студия»,4,2
11214,1,190100.0,43500,Система быстрых платежей: Эквайринг ComfortBoo...,2023-11-11 17:51:55,2024-02-16 15:00:00,5,2024-02-21 12:00:00,Официальный сайт,Коттедж с 2 спальнями,4,1
11215,1,42300.0,42300,Банк. карта: Банк Россия (банк. карта),2022-02-09 02:23:14,2022-02-09 15:00:00,1,2022-02-10 12:00:00,Официальный сайт,Коттедж с 3 спальнями,5,1
11216,1,27900.0,27900,Банк. карта: Банк Россия (банк. карта),2023-01-21 09:10:17,2023-01-28 15:00:00,1,2023-01-29 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1


In [275]:
res=model.predict(tdf)
display(res)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [276]:
hist, bin_edges = np.histogram(res, bins=2)
print(f"Гистограмма: {hist}")
print(f"Границы интервалов: {bin_edges}")

Гистограмма: [10059  1159]
Границы интервалов: [0.  0.5 1. ]


In [277]:
catboost_res_dir = os.path.join(root_dir, 'catboost_res')

In [278]:
res_df = pd.DataFrame(res)
res_df.to_csv(os.path.join(catboost_res_dir,"res2.csv"),index = False, header=None)

# Learn on all data

In [279]:
model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=True)

In [280]:
categorical_features = ["Способ оплаты", "Источник","Категория номера"]

In [281]:
df = pd.read_excel(train_path)
target = df['Дата отмены'].notna().astype(int)
df = df.drop(["Unnamed: 0","№ брони", "Дата отмены","Статус брони"], axis=1)
display(df.head(),df.shape)

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Номер «Стандарт»,2,1
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Номер «Студия»,2,4
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Номер «Стандарт»,1,3
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Номер «Люкс»,4,1


(26174, 12)

In [282]:
tdf_path = os.path.join(data_dir, 'test.xlsx')
display(tdf_path)
tdf = pd.read_excel(tdf_path)
display(tdf.head())
tdf = tdf.drop(["Unnamed: 0","№ брони"], axis=1)
display(tdf.head())

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack\\data\\test.xlsx'

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,0,20231129-16563-238946689,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,20221219-7491-174959103,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,2,20221211-6634-172724329,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,3,20230821-6634-212247350,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,4,20230326-6634-189784563,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1


,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1


In [283]:
train_df = df 
test_df = tdf

In [284]:
train_pool = Pool(train_df, target, cat_features=categorical_features)

In [285]:
model.fit(train_pool)

0:	learn: 0.6418154	total: 23.3ms	remaining: 2.31s
1:	learn: 0.5997459	total: 43ms	remaining: 2.11s
2:	learn: 0.5648227	total: 64.6ms	remaining: 2.09s
3:	learn: 0.5358785	total: 85.8ms	remaining: 2.06s
4:	learn: 0.5109153	total: 108ms	remaining: 2.04s
5:	learn: 0.4904515	total: 130ms	remaining: 2.04s
6:	learn: 0.4649176	total: 154ms	remaining: 2.05s
7:	learn: 0.4435076	total: 177ms	remaining: 2.03s
8:	learn: 0.4249021	total: 196ms	remaining: 1.98s
9:	learn: 0.4092383	total: 218ms	remaining: 1.96s
10:	learn: 0.3959147	total: 238ms	remaining: 1.92s
11:	learn: 0.3850640	total: 261ms	remaining: 1.91s
12:	learn: 0.3747259	total: 285ms	remaining: 1.91s
13:	learn: 0.3664597	total: 309ms	remaining: 1.9s
14:	learn: 0.3592835	total: 342ms	remaining: 1.94s
15:	learn: 0.3531930	total: 368ms	remaining: 1.93s
16:	learn: 0.3477506	total: 409ms	remaining: 1.99s
17:	learn: 0.3434238	total: 430ms	remaining: 1.96s
18:	learn: 0.3396112	total: 450ms	remaining: 1.92s
19:	learn: 0.3358394	total: 472ms	remain

In [286]:
res=model.predict(tdf)
display(res)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [287]:
res_df = pd.DataFrame(res)
res_df.to_csv(os.path.join(catboost_res_dir,"res_all1.csv"),index = False, header=None)

## Another preproc
### Hotel to [0,1]

In [288]:
categorical_features = ["Способ оплаты", "Источник","Категория номера"]

In [ ]:
df = pd.read_excel(train_path)
target = df['Дата отмены'].notna().astype(int)
df = df.drop(["Unnamed: 0","№ брони", "Дата отмены","Статус брони"], axis=1)
display(df.head(),df.shape)

In [ ]:
df['Гостиница']=(df['Гостиница'] >= 3).astype(int)

In [ ]:
tdf_path = os.path.join(data_dir, 'test.xlsx')
display(tdf_path)
tdf = pd.read_excel(tdf_path)
display(tdf.head())
tdf = tdf.drop(["Unnamed: 0","№ брони"], axis=1)
display(tdf.head())

In [ ]:
train_df = df
test_df = tdf
train_pool = Pool(train_df, target, cat_features=categorical_features)

In [ ]:
model.fit(train_pool)

In [ ]:
res=model.predict(tdf)
display(res)

In [ ]:
res_df = pd.DataFrame(res)
res_df.to_csv(os.path.join(catboost_res_dir,"Hotel.csv"),index = False, header=None)